Enter Path to the folder you wish to process:

In [1]:
mainFolder = "/data/Johannes/2020_09_03/"
# mainFolder = "/data/Marjan/MB2020_lifs_1/"

## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
# import numpy as np
# import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
from IPython.display import display
import pandas as pd
from islets.Recording import Recording, parse_leica
from islets.utils import get_filterSizes

In [2]:
recordings = []
for cur,ds,fs in os.walk(mainFolder):
    for f in fs:
        if not (f.endswith(".lif") or f.endswith(".nd2")):
            continue
        path = os.path.join(cur,f)
        recordings += [path]
recordings = sorted(recordings)

In [3]:
status = []
ilifs = 0
for pathToRecording in recordings:
    print ("#"*20, pathToRecording)
    rec = Recording(pathToRecording)
    recType = "Nikon" if pathToRecording.endswith(".nd2") else "Leica"

    if recType=="Leica":
        sers = parse_leica(rec)
    else:
        sers = ["all"]

    analysisFolder = os.path.join(rec.folder, rec.Experiment+"_analysis")
#     if not os.path.isdir(analysisFolder):
#         os.makedirs(analysisFolder)
        
    for ser in sers:
        md = pd.Series()
        md["path"] = pathToRecording
        md["exp"] = os.path.split(pathToRecording)[-1]
        md["series"] = ser
        try:
            rec.import_series(ser, onlyMeta=True)
        except:
            print (f"could not import {ser}")
            status += [md]
            continue
            
        saveDir = os.path.join(analysisFolder, ser)
        for k,v in rec.Series[ser]["metadata"].items(): md[k] = v
        fs = get_filterSizes(md.pxSize)
#         md["filter sizes"] = fs
        movieFilename = os.path.join(saveDir, rec.Experiment+"_"+ser+".mp4")
        md["movie"] = os.path.isfile(movieFilename)
        if md["movie"]:
            md["movie size"] = os.path.getsize(movieFilename)
        for k in ["bit depth", "Start time", "End time", "individual Series","Name"]:
            try:    del md[k]
            except: pass
        pklsDone = {}
        imgsDone = {}
        for fsize in fs:
            pickleFile = os.path.join(saveDir, ".".join(map(str,fsize))+"_rois.pkl")
            imageFile = os.path.join(saveDir, ".image_"+".".join(map(str,fsize))+".png")
            pickleThere = os.path.isfile(pickleFile)
            imageThere = os.path.isfile(imageFile)
#             if pickleThere:
#                 purge = True
#                 try:
#                     with open(pickleFile,"rb") as f:
#                         rr = pickle.load(f)
#                     purge = "trace" not in rr.df.columns
#                     if not purge: lastGood = pickleFile
#                 except:
#                     purge = True
#                 if purge:
#                     os.remove(pickleFile)
#                     if imageThere: os.remove(imageFile)
            pklsDone[fsize] = pickleThere
            imgsDone[fsize] = imageThere
        md["pickles"] = pklsDone
        md["images"] = pklsDone
#         md["allDone"] = np.array(list(pklsDone.values())).all() and md["movie done"]
        status += [dict(md.items())]
    ilifs +=1
#     if ilifs>3:
#         break

status = pd.DataFrame(status)

#################### /data/Johannes/2020_09_03/Experiment007a.lif
#################### /data/Johannes/2020_09_03/Experiment007b.lif
#################### /data/Johannes/2020_09_03/Experiment007c.lif
#################### /data/Johannes/2020_09_03/Experiment007d.lif
#################### /data/Johannes/2020_09_03/Experiment007e.lif


In [4]:
for c in status.columns:
    if "Size" in c:
        status[c.replace("Size","")] = status[c]
        del status[c]

status["location"] = status.path.apply(lambda xi: os.path.split(xi)[0])

status["Duration"] = status["Duration"].apply(str).apply(lambda xi: xi.split()[-1].split(".")[0] )

firstCols = ["exp", "series","movie","pickles","images",]
status = status[firstCols+[c for c in status.columns if c not in firstCols]]

lastCols = ["path"]
status = status[[c for c in status.columns if c not in lastCols]+lastCols]

del status["pxUnit"]

In [5]:
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

In [10]:
def cellTransform(cellvalue, id=None):
    import numpy as np
#     print (type(cellvalue))
    if issubclass(type(cellvalue),(str,int,float,np.int_)):
        out = cellvalue
    elif issubclass(type(cellvalue),(bool,np.bool_)):
        out = dcc.Checklist(id=id,
                             options=[{"label":"✔" if cellvalue else "✘","value":None}],
                             value=[None]*(1-int(cellvalue)),
                             labelStyle={"color":"green" if cellvalue else "red"}
                            )
    elif issubclass(type(cellvalue),(dict,)):
        out = dcc.Checklist(id=id,
                             options=[{"label":(str(k).replace(" ","")+("✔" if cellvalue[k] else "✘")), "value":k} for k in cellvalue],
                             value = [k for k in cellvalue if not cellvalue[k]],
#                              style={"height":"10px",},
                             labelStyle={
                                 "width":"100px",
                                 "display":"block"
                                        }
                            )
    else:
        out = str(cellvalue)
    return html.Div(out,style={
                                 "font-family":"monospace",
                                 "font-size":"14px",
                             },)
#     if issubclass(type(value),str):

In [11]:
def generate_table(dataframe, max_rows=100):
    rows = []
    for i in range(min(len(dataframe), max_rows)):
        row = []
        for col in dataframe.columns:
            value = dataframe.iloc[i][col]
#             row.append(html.Td(value if issubclass(type(value),str) else dcc.Checklist(options=[{"label":"item"}]),
# #                                style=style
#                               ))
            row.append(html.Td(cellTransform(value, id=f"table-{col}-{i}")))
        rows.append(html.Tr(row))

    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        rows) 

In [8]:
import json
from collections import OrderedDict
from copy import deepcopy

In [12]:
htmltable = generate_table(status.iloc[:,:-1])

app = JupyterDash(__name__, width=1000)

app.layout = html.Div([
    html.H1("Table"),
    html.Div(htmltable,style={"width":"1000px"}),
    html.Br(),
    html.Div(id="marked",children="------------",),
],style={"font-family":"Arial"})
# app.scripts.append_css({'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

ixOrder = OrderedDict([(f"table-{col}-{i}",{"col":col,"index":i}) \
        for col in ["movie","pickles","images"]\
        for i in status.index])
@app.callback(
    Output("marked","children"),
    [Input(k,"value") for k in ixOrder]
)
def see(*manyinputs):
    print (manyinputs)
    out = deepcopy(ixOrder)
    for k,v in zip(out, manyinputs):
        out[k]["value"] = v
#     return json.dumps(dict(zip(ixOrder.keys(),manyinputs)), indent=2)
#     return html.Div(json.dumps(out, indent=2), 
#              style={
#             "font-family":"Courier New",
#             "font-size":"80%",
#             'border': 'thin lightgrey solid',
#             "width":"400px",
#             "height":"300px",
#             'overflowX': 'scroll',
#             'overflowY': 'scroll',}
#                    )
    out = list(out.values())
    toDo = {}
    for el in out:
        if len(el["value"])<1: continue
        col = el["col"]
        i = el["index"]
        if col not in toDo:
            toDo[col] = []
        add = {
            "rec":status.loc[i,"path"],
            "series":status.loc[i,"series"]
        }
        if col!="movie":
            add["filterSizes"] = list(map(tuple,el["value"]))
            add["filterSizes"] = str(add["filterSizes"])
        toDo[col] += [add]
    return html.Div(json.dumps(toDo, indent=2), 
         style={
        "font-family":"Courier New",
        "font-size":"80%",
        'border': 'thin lightgrey solid',
        "width":"400px",
        "height":"300px",
        'overflowX': 'scroll',
        'overflowY': 'scroll',}
               )
#     out = pd.DataFrame(out)
#     ix = out.query("value=='movie'").index
#     out.loc[ix,"value"] = [len(out.loc[i,"value"]) for i in ix]
#     out[]["value"] = 
#     return generate_table(out)

app